# Generating Seq2seq dataset

## noting the versions 

In [14]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib

def check_version(module, expected_version):
    actual_version = module.__version__
    if actual_version == expected_version:
        print(f"{module.__name__} version is correct: {actual_version}")
    else:
        print(f"{module.__name__} version is incorrect: {actual_version} (expected: {expected_version})")

check_version(np, "1.21.0")
check_version(pd, "1.3.0")
check_version(nx, "2.5.1")
check_version(matplotlib, "3.4.2")

numpy version is incorrect: 1.23.0 (expected: 1.21.0)
pandas version is incorrect: 1.4.3 (expected: 1.3.0)
networkx version is incorrect: 2.8.4 (expected: 2.5.1)
matplotlib version is incorrect: 3.5.2 (expected: 3.4.2)


# helpers

In [91]:
import random
import pandas as pd
import json


def node_specification(data_topologi):
    Speed =[]
    ram =[]
    storage=[]
    id_node=[]
    bandwith_node = []

    for n in data_topologi['nodes']:
        Speed.append(n['Speed'])
        ram.append(n['RAM'])
        storage.append(n['Storage'])
        id_node.append(n['id'])
        bandwith_node.append(75000)

    df_nodes = pd.DataFrame(list(zip(id_node, Speed, ram, storage, bandwith_node)), columns =['id', 'Speed', 'RAM', 'Storage', 'Bandwith'])
    return df_nodes

def app_specification(data_app):
    name_app_module=[]
    ram_module=[]
    storage_module= []
    name_module=[]

    name_app_message=[]
    bytes_size=[]
    instruction=[]
    name_message=[]

    deadline=[]

    for k in data_app:
        name_app = k['name']
        for l in k['module']:
            s = l['name']
            namanya = s.replace("'", "")
            ram_module.append(l['RAM'])
            storage_module.append(l['Storage'])
            name_module.append(namanya)
            name_app_module.append(name_app)
            deadline.append(k['deadline'])

        for m in k['message']:
            name_app_message.append(name_app)
            name_message.append(m['name'])
            bytes_size.append(m['bytes'])
            instruction.append(m['instructions'])

    df_modules = pd.DataFrame(list(zip(name_app_module, name_module, ram_module, storage_module ,deadline)), columns =['app', 'Module', 'RAM', 'Storage', 'deadline'])
    df_message = pd.DataFrame(list(zip(name_message, bytes_size, instruction)), columns =['Message', 'bytes', 'instruction'])
    final_app = pd.concat([df_modules, df_message], axis=1)
    return final_app

def create_dict(topo):
    new_dict = dict()
    source = []
    for i in topo['links']:
        source.append(i['source'])
        if i['source'] in new_dict:
            # append the new number to the existing array at this slot
            new_dict[i['source']].append(i['target'])
            if i['target'] in new_dict:
                new_dict[i['target']].append(i['source'])
            else:
                new_dict[i['target']] = [i['source']]
        else:
            # create a new array in this slot
            new_dict[i['source']] = [i['target']]
    return new_dict, source

def checkthisout(place, appp, topo):
    # nanti masukin pathnya aja
    ## APP input


    f = open(str(place))
    data_placement = json.load(f)

    df_nodes = topo


    #framing placement
    app =[]
    Mod =[]
    node = []

    for n in data_placement['initialAllocation']:
        app.append(n['app'])
        Mod.append(n['module_name'])
        node.append(n['id_resource'])
    data_popo = pd.DataFrame(list(zip(app, Mod, node)), columns =['App', 'Mod', 'node'])

    df_apps = appp
    muatan = []
    for index, row in data_popo.iterrows():
        current_mod = df_apps.loc[(df_apps['app'] == str(row['App']))&(df_apps['Module'] == str(row['Mod']))]
        df_nodes.loc[row['node'], 'Storage'] = int(df_nodes.loc[row['node'], 'Storage']) - int(current_mod['Storage'])
        # df_nodes.loc[row['node'], 'IPT'] = int(df_nodes.loc[row['node'], 'IPT']) - int(current_mod['Instruction'])

        # df_nodes.loc[row['node'], 'Bandwith'] = int(df_nodes.loc[row['node'], 'Bandwith']) - int(current_mod['Bandwith'])

        # if((df_nodes.loc[row['node'], 'IPT'] <= 0) or (df_nodes.loc[row['node'], 'RAM'] <= 0) or (df_nodes.loc[row['node'], 'Bandwith'] <= 0)):
        #     muatan.append('Tidak Muat')
        # else:
        #     muatan.append('Muat')

        if(df_nodes.loc[row['node'], 'RAM'] <= 0):
            muatan.append('Tidak Muat')
        else:
            muatan.append('Muat')
    return muatan

def createlistofplacement(linkplacement):
    f = open(str(linkplacement))
    data_placement = json.load(f)

    #framing placement
    app =[]
    Mod =[]
    node = []

    for n in data_placement['initialAllocation']:
        app.append(n['app'])
        Mod.append(n['module_name'])
        node.append(n['id_resource'])
    data_popo = pd.DataFrame(list(zip(app, Mod, node)), columns =['App', 'Mod', 'node'])
    return data_popo

In [32]:
def nodes_extract_to_dataframe(filepath):

    try:
        # Reading and parsing the JSON file
        with open(filepath, 'r') as file:
            data = json.load(file)

        # Extracting the 'nodes' data
        nodes_data = data['nodes']

        # Creating a DataFrame from the nodes data
        return pd.DataFrame(nodes_data)
    except Exception as e:
        # In case of any error, return the error message
        return str(e)

def module_json_to_dataframe(json_file_path):
    # Read the JSON file
    with open(json_file_path, 'r') as f:
        data = json.load(f)

    # Prepare an empty list to store rows
    rows = []

    # Iterate over each application
    for app in data:
        app_id = app['id']
        numberofmodule = app['numberofmodule']

        # Iterate over each module and message
        for module, message in zip(app['module'], app['message']):
            row = {
                'Application ID': str(app_id),
                'numberofmodule': numberofmodule,
                'Module ID': module['id'],
                'Module Name': module['name'],
                'Required RAM': module['RAM'],
                'Required Storage': module['Storage'],
                'Message ID': message['id'],
                'Message Name': message['name'],
                'Bytes': message['bytes'],
                'Instructions': message['instructions']
            }
            rows.append(row)

    # Convert the list of rows into a DataFrame
    df = pd.DataFrame(rows)

    return df

def placement_json_to_dataframe(json_file_path):
    with open(json_file_path, 'r') as f:
        data = json.load(f)
    rows = []
    for placement in data['initialAllocation']:
        rows.append({
            'Module Name': str(placement['module_name']),
            'Application ID': str(placement['app']),
            'Node ID': placement['id_resource']
        })
    return pd.DataFrame(rows)


# Main function to check placement validity
def check_placement_validity(topology_file, application_file, placement_file):
    # Parse the JSON files into DataFrames
    nodes_df = nodes_extract_to_dataframe(topology_file)
    application_df = module_json_to_dataframe(application_file)
    placement_df = placement_json_to_dataframe(placement_file)


    # # # Diagnostic prints to check DataFrame structures
    # print("Placement DataFrame Columns:", placement_df.columns)
    # print("Application DataFrame Columns:", application_df.columns)
    # print("Data types in Placement DataFrame:", placement_df.dtypes)
    # print("Data types in Application DataFrame:", application_df.dtypes)

    # Merge and calculate Storage usage
    placement_app_merged = pd.merge(placement_df, application_df, on=['Module Name', 'Application ID'], how='left')
    node_ram_usage = placement_app_merged.groupby('Node ID')['Required Storage'].sum().reset_index()
    nodes_df.rename(columns={'Storage': 'Available Storage'}, inplace=True)

    # Compare available and required Storage
    node_Storage_comparison = pd.merge(nodes_df, node_ram_usage, left_on='id', right_on='Node ID', how='left')
    node_Storage_comparison['Required Storage'].fillna(0, inplace=True)
    node_Storage_comparison['Storage_Sufficient'] = node_ram_comparison['Available Storage'] >= node_ram_comparison['Required Storage']

       # Check if all nodes have sufficient Storage and print a message if so
    if node_Storage_comparison['Storage_Sufficient'].all():
        print("All is good.")
        return node_Storage_comparison[['id', 'Available Storage', 'Required Storage', 'Storage_Sufficient']]
    else:
        print("Something not fit")
        return node_Storage_comparison[['id', 'Available Storage', 'Required Storage', 'Storage_Sufficient']]



# HopAware

In [92]:
import random
import pandas as pd
import json
import os

# Helper function to safely get the next node choice
def get_next_pilihan(current_pilihan, new_dict):
    """Safely gets a random neighbor or defaults to cloud (100)."""
    try:
        # Ensure key is of the correct type for new_dict (assuming int)
        current_pilihan_key = int(current_pilihan)

        if current_pilihan_key == 100:
            return 100 # Stay in cloud if already there

        neighbors = new_dict.get(current_pilihan_key, [])
        if neighbors:
            # Ensure neighbors are suitable types before returning
            chosen_neighbor = random.choice(neighbors)
            return int(chosen_neighbor) # Assuming neighbors are node IDs
        else:
            # No neighbors found for the current node, default to cloud
            # print(f"Warning: Node {current_pilihan_key} has no neighbors in new_dict. Defaulting to cloud.")
            return 100
    except (ValueError, TypeError) as e:
        # print(f"Warning: Error processing pilihan {current_pilihan} or its neighbors: {e}. Defaulting to cloud.")
        return 100
    except Exception as e: # Catch other potential errors
        # print(f"Unexpected error getting next pilihan for {current_pilihan}: {e}. Defaulting to cloud.")
        return 100

# Renaming the function to match the user's selection
def create_placementold_newformat(data_pop, new_dict, app_spec, node_spec, popul, app, save_folder, hopnumber):
    JSONfile = {"initialAllocation": []}
    allocation = [] # List to store allocation dicts for JSON
    output_lines = [] # List to store formatted strings for TXT
    order = 0  # Initialize order counter for TXT output

    # Create a local copy of node Storage to modify during placement simulation for this run.
    # This prevents modifications affecting subsequent calls or applications if node_spec is reused.
    # Ensure index is usable (assuming 'id' column was used or index is already int)
    try:
        if not pd.api.types.is_integer_dtype(node_spec.index):
             # If index isn't integer, try using 'id' column if it exists and is suitable
             if 'id' in node_spec.columns and pd.api.types.is_numeric_dtype(node_spec['id']):
                 node_spec = node_spec.set_index('id', drop=False) # Keep id column if needed elsewhere
             else:
                 # Fallback or raise error if no suitable integer index can be found
                 print("Warning: node_spec index is not integer, Storage lookup might fail.")
        # Use 'Storage' instead of 'RAM'
        local_node_spec_storage = node_spec['Storage'].astype(int).copy()
    except Exception as e:
        # Use 'Storage' instead of 'RAM'
        print(f"Error preparing local node Storage spec: {e}. Aborting placement.")
        return False


    for i in data_pop['sources']:
        source_node_id = i['id_resource']
        applik = i['app'] # Application identifier
        # Get lambda value, default to 'N/A' if not found
        lambda_val = i.get('lambda', 'N/A')

        # --- Determine initial starting node ('pilihan') based on hopnumber ---
        try:
            source_node_int = int(source_node_id)
            if hopnumber == 3:
                pilihan = get_next_pilihan(source_node_int, new_dict) # Start from a neighbor
                # print(f"HOP3: Initial pilihan {pilihan} (neighbor of {source_node_int})")
            elif hopnumber == 2:
                pilihan = source_node_int # Start from the source node itself
                # print(f"HOP2: Initial pilihan {pilihan} (source node)")
            else:
                # print(f"Warning: Unsupported hopnumber {hopnumber}. Defaulting to HOP2 logic.")
                pilihan = source_node_int
        except (ValueError, TypeError) as e:
            # print(f"Warning: Invalid source node ID {source_node_id}: {e}. Starting at cloud.")
            pilihan = 100 # Default to cloud if source ID is invalid
        except Exception as e:
             # print(f"Unexpected error determining initial pilihan for source {source_node_id}: {e}. Starting at cloud.")
             pilihan = 100

        # --- Filter application specification ---
        try:
            # Ensure comparison works (e.g., both strings or both ints)
            # Assuming app_spec['app'] might be int/str, applik is likely str from JSON
            current_app = app_spec[app_spec['app'].astype(str) == str(applik)]
        except Exception as e:
            print(f"Error filtering app_spec for app {applik}: {e}. Skipping application.")
            continue

        if current_app.empty:
             # print(f"Warning: No modules found for application {applik} in app_spec. Skipping.")
             continue

        # --- Initialize for TXT output for this application ---
        module_count = len(current_app)
        placement_line = [] # Stores node placements for TXT format (e.g., "n5", "n100")
        # Initialize spec_line with order, module count, and lambda
        spec_line = [f"o{order}", f"m{module_count}", f"l{lambda_val}"] # Stores spec info for TXT format
        failed_nodes = []  # Stores failed nodes for TXT format (e.g., "f3")

        # --- Iterate through modules of the current application ---
        for index, row in current_app.iterrows():
            module_name = str(row.get('Module', f'UnknownModule_{index}')) # Safely get module name
            try:
                # Use 'Storage' instead of 'RAM'
                required_storage = int(row['Storage'])
            except (KeyError, ValueError, TypeError) as e:
                # Use 'Storage' instead of 'RAM'
                print(f"Warning: Invalid Storage for module {module_name} (App {applik}): {row.get('Storage', 'N/A')}. Skipping module. Error: {e}")
                placement_line.append("n?_invalid_storage") # Indicate skip in TXT
                spec_line.append(f"r?_invalid") # Indicate skip in TXT spec (keep 'r' prefix)
                continue # Skip this module

            # Use 'Storage' instead of 'RAM'
            spec_line.append(f"r{required_storage}")  # Add required Storage to TXT spec line

            ketemu = False # Flag: Found placement for this module?
            count_first = 0 # Outer loop counter (placement attempt sequences)

            # Store the node pilihan *before* starting the search for this module
            current_pilihan_state = pilihan # This holds the node to try *next*

            while count_first < 3 and not ketemu:
                count = 0 # Inner loop counter (hops within a sequence)
                node_to_evaluate = current_pilihan_state # Node to check in this hop

                while count < 5: # Try up to 5 hops
                    string1 = {} # To store JSON allocation info for this module

                    try:
                        node_to_evaluate_int = int(node_to_evaluate)
                    except (ValueError, TypeError):
                        # print(f"Warning: Invalid node ID {node_to_evaluate} during search. Treating as cloud.")
                        node_to_evaluate_int = 100

                    # --- Check Cloud ---
                    if node_to_evaluate_int == 100:
                        # print(f"Placing module {module_name} (App {applik}) on Cloud (100).")
                        string1 = {"module_name": module_name, "app": str(applik), "id_resource": 100}
                        placement_line.append("n100")
                        current_pilihan_state = 100 # Next module search also starts at cloud
                        ketemu = True
                        break # Exit inner loop (count < 5)

                    # --- Check Node Resources ---
                    try:
                        # Use 'Storage' instead of 'RAM'
                        available_storage = int(local_node_spec_storage.loc[node_to_evaluate_int])
                        # Use 'Storage' instead of 'RAM'
                        storage_after_placement = available_storage - required_storage

                        # Use 'Storage' instead of 'RAM'
                        if storage_after_placement >= 0:
                            # Placement success on this node
                            # Use 'Storage' instead of 'RAM'
                            # print(f"Placing module {module_name} (App {applik}) on Node {node_to_evaluate_int} (Storage {available_storage} -> {storage_after_placement}).")
                            string1 = {"module_name": module_name, "app": str(applik), "id_resource": node_to_evaluate_int}
                            placement_line.append(f"n{node_to_evaluate_int}")

                            # Update the local Storage copy
                            # Use 'Storage' instead of 'RAM'
                            local_node_spec_storage.loc[node_to_evaluate_int] = storage_after_placement

                            # Determine starting node for the *next* module's search
                            current_pilihan_state = get_next_pilihan(node_to_evaluate_int, new_dict)
                            # print(f"Next module search will start near node {current_pilihan_state}")

                            ketemu = True
                            break # Exit inner loop (count < 5)
                        else:
                            # Insufficient Storage on this node
                            # Use 'Storage' instead of 'RAM'
                            # print(f"Node {node_to_evaluate_int} insufficient Storage ({available_storage} < {required_storage}) for {module_name}. Trying next hop.")
                            failed_node_str = f"f{node_to_evaluate_int}"
                            if failed_node_str not in failed_nodes: # Avoid duplicates in TXT
                                failed_nodes.append(failed_node_str)

                            # Move to the next node in the hop sequence
                            node_to_evaluate = get_next_pilihan(node_to_evaluate_int, new_dict)
                            # print(f"Hopping to node {node_to_evaluate}")

                    except (KeyError):
                        # Use 'Storage' instead of 'RAM'
                        # print(f"Warning: Node {node_to_evaluate_int} not found in local Storage spec. Treating as failed.")
                        failed_node_str = f"f{node_to_evaluate_int}_not_found"
                        if failed_node_str not in failed_nodes: failed_nodes.append(failed_node_str)
                        node_to_evaluate = get_next_pilihan(node_to_evaluate_int, new_dict) # Try neighbor
                    except (ValueError, TypeError) as e:
                         # Use 'Storage' instead of 'RAM'
                         print(f"Warning: Error processing Storage for node {node_to_evaluate_int}: {e}. Treating as failed.")
                         # Use 'Storage' instead of 'RAM'
                         failed_node_str = f"f{node_to_evaluate_int}_storage_error"
                         if failed_node_str not in failed_nodes: failed_nodes.append(failed_node_str)
                         node_to_evaluate = get_next_pilihan(node_to_evaluate_int, new_dict) # Try neighbor

                    count += 1 # Increment inner loop counter (hops)
                # --- End of inner loop (while count < 5) ---

                if ketemu:
                    break # Exit outer loop (count_first < 3)
                else:
                    # Failed to find placement within 5 hops starting from where we left off.
                    # The example logic seems to just continue from the last 'pilihan' state.
                    # The outer loop (count_first) in the example seems more like a safeguard against infinite loops
                    # rather than restarting the search from different points. Let's follow that.
                    # If the inner loop finished, current_pilihan_state holds the last node tried or hopped to.
                    # We just continue the outer loop check. If count_first reaches 3, we fail.
                    # print(f"Placement attempt sequence {count_first + 1} failed for module {module_name}.")
                    pass # Just let the outer loop increment

                count_first += 1 # Increment outer loop counter
            # --- End of outer loop (while count_first < 3) ---

            # --- Handle placement failure after all attempts ---
            if not ketemu:
                # print(f"Failed to place module {module_name} (App {applik}) after {count_first} attempts. Placing on Cloud (100).")
                string1 = {"module_name": module_name, "app": str(applik), "id_resource": 100}
                placement_line.append("n100")
                current_pilihan_state = 100 # Ensure next module search starts at cloud

            # --- Add allocation result to JSON list ---
            if string1: # Only append if placement was determined
                allocation.append(string1)
            else:
                 # This case should ideally not happen if failure defaults to cloud
                 print(f"Error: Module {module_name} (App {applik}) finished without allocation decision.")
                 placement_line.append("n?_error") # Indicate error in TXT

            # Update the main 'pilihan' state for the next module based on where this one ended up
            pilihan = current_pilihan_state

        # --- Finalize TXT output lines for this application ---
        spec_line.append(f"n{source_node_id}") # Add source node ID
        spec_line.extend(failed_nodes) # Add recorded failed nodes
        output_lines.append(" ".join(spec_line))
        output_lines.append(" ".join(placement_line))

        order += 1  # Increment order for the next application

    # --- End of application loop (for i in data_pop['sources']) ---

    # --- Save JSON output ---
    JSONfile["initialAllocation"] = allocation
    json_object = json.dumps(JSONfile, indent=2)
    json_filename = f"placement\D100_P{popul}_A{app}_hop{hopnumber}.json"
    json_filepath = os.path.join(save_folder, json_filename)
    try:
        os.makedirs(save_folder, exist_ok=True)
        with open(json_filepath, "w") as outfile: # Write mode for JSON
            outfile.write(json_object)
        # print(f"JSON allocation saved to {json_filepath}")
    except IOError as e:
        print(f"Error writing JSON file {json_filepath}: {e}")
        # Consider returning False or raising exception

    # --- Save TXT output ---
    txt_filename = f"D100_P{popul}_A{app}_hop{hopnumber}.txt"
    txt_filepath = os.path.join(save_folder, txt_filename)
    try:
        os.makedirs(save_folder, exist_ok=True)
        # Use append mode 'a' for the TXT file
        with open(txt_filepath, "a") as outfile:
            # Add a newline before appending if the file already exists and is not empty
            if os.path.exists(txt_filepath) and os.path.getsize(txt_filepath) > 0:
                 outfile.write("\n")
            outfile.write("\n".join(output_lines)) # Write the lines for the current run
        # print(f"TXT decision data appended to {txt_filepath}")
    except IOError as e:
        print(f"Error appending TXT file {txt_filepath}: {e}")
        # Consider returning False or raising exception

    return True # Indicate success (or partial success if errors occurred but were handled)

## testing

In [53]:
import json
import time
import pandas as pd

execution_times = []
methodname = "hop3_withfailure"
hopnumba = 2

pop = 0
app = 10

f = open(fr'C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\topology\100m2.json')
# a dictionary
data = json.load(f)
# print(f)

f = open(fr"C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\population\100m2_P{pop}_E{app}.json")
data_pop = json.load(f)
# print(f)

f = open(fr'C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\application\E{app}_{pop}.json')
data_app = json.load(f)
        # print(f)

new_dict, source = create_dict(data)
app_spec = app_specification(data_app)
node_spec = node_specification(data)
source = list(dict.fromkeys(source))
pop_id = pop
app_id = app

save_folder = fr"C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\training_data"
# Time the execution of create_placementold
start_time = time.time()
create_placementold_newformat(data_pop, new_dict, app_spec, node_spec, pop_id, app_id, save_folder, hopnumber=hopnumba)
end_time = time.time()

execution_time = end_time - start_time
execution_times.append({'pop': pop, 'app': app, 'execution_time': execution_time, 'methodname': methodname})

# Create DataFrame from execution times
df_execution_times = pd.DataFrame(execution_times)

# Save DataFrame to CSV
df_execution_times.to_csv(f"execution_times_{methodname}.csv", index=False)

In [41]:
app_spec

,app,Module,RAM,Storage,deadline,Message,bytes,instruction
0,0,Mod0,6,4,26658,M.USER.APP.0,1652773,59402
1,0,Mod1,1,4,26658,M1,3046134,46009
2,1,Mod0,3,6,6664,M.USER.APP.1,4356231,29732
3,1,Mod1,4,3,6664,M1,1626814,26718
4,1,Mod2,1,5,6664,M2,2194439,21605
...,...,...,...,...,...,...,...,...
574,99,Mod5,3,2,13116,M5,2926084,55616
575,99,Mod6,2,2,13116,M6,4266991,56826
576,99,Mod7,5,6,13116,M7,4234856,20269
577,99,Mod8,6,5,13116,M8,2473891,36021


## hop3

In [93]:
import json
import time
import pandas as pd

execution_times = []
methodname = "hop3_withfailure"

for pop in range(10):
    for app in range(10):

        f = open(fr'C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\topology\100m2.json')
        # a dictionary
        data = json.load(f)
        # print(f)

        f = open(fr"C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\population\100m2_P{pop}_D100.json")
        # a dictionary
        data_pop = json.load(f)
        # print(f)

        f = open(fr'C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\application\D100_{app}.json')
        data_app = json.load(f)
        # print(f)

        new_dict, source = create_dict(data)
        app_spec = app_specification(data_app)
        node_spec = node_specification(data)
        source = list(dict.fromkeys(source))
        pop_id = pop
        app_id = app

        save_folder = fr"C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\training_data"
        # Time the execution of create_placementold
        start_time = time.time()
        create_placementold_newformat(data_pop, new_dict, app_spec, node_spec, pop_id, app_id, save_folder, hopnumber=3)
        end_time = time.time()

        execution_time = end_time - start_time
        execution_times.append({'pop': pop, 'app': app, 'execution_time': execution_time, 'methodname': methodname})

# Create DataFrame from execution times
df_execution_times = pd.DataFrame(execution_times)

# Save DataFrame to CSV
df_execution_times.to_csv(f"execution_times_{methodname}.csv", index=False)

## hop2

In [94]:
import json
import time
import pandas as pd

execution_times = []
methodname = "hop2_withfailure"

for pop in range(10):
    for app in range(10):

        f = open(fr'C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\topology\100m2.json')
        # a dictionary
        data = json.load(f)
        # print(f)

        f = open(fr"C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\population\100m2_P{pop}_D100.json")
        # a dictionary
        data_pop = json.load(f)
        # print(f)

        f = open(fr'C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\application\D100_{app}.json')
        data_app = json.load(f)
        # print(f)

        new_dict, source = create_dict(data)
        app_spec = app_specification(data_app)
        node_spec = node_specification(data)
        source = list(dict.fromkeys(source))
        pop_id = pop
        app_id = app

        save_folder = fr"C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\training_data"
        # Time the execution of create_placementold
        start_time = time.time()
        create_placementold_newformat(data_pop, new_dict, app_spec, node_spec, pop_id, app_id, save_folder, hopnumber=2)
        end_time = time.time()

        execution_time = end_time - start_time
        execution_times.append({'pop': pop, 'app': app, 'execution_time': execution_time, 'methodname': methodname})

# Create DataFrame from execution times
df_execution_times = pd.DataFrame(execution_times)

# Save DataFrame to CSV
df_execution_times.to_csv(f"execution_times_{methodname}.csv", index=False)

# firstfit_hopware

In [96]:
import pandas as pd
import json
import random
import os

# Modified find_suitable_node to return failed attempts for logging
# Replaced 'RAM' with 'Storage'
def find_suitable_node(row, node_spec, current_resource_id, adjacency_dict, cloud_node_id):
    """
    Find a suitable node for module placement based on Storage with a fallback to cloud node.
    Now also returns a list of nodes that failed due to insufficient Storage during this search.
    """
    failed_nodes_for_this_call = [] # Track nodes that failed in this specific search
    original_start_node = current_resource_id # Keep track of where this search started

    for retry in range(5):  # retry up to 5 times
        search_path_this_retry = [] # Track path within a retry to avoid adding duplicates quickly
        temp_resource_id = original_start_node if retry == 0 else current_resource_id # Start fresh or continue from last random hop

        for attempt in range(5):  # try up to 5 times per retry
            available_storage = -1 # Default to -1 if node ID is invalid or not found
            node_to_check = int(temp_resource_id) # Ensure integer ID

            if node_to_check in node_spec.index:
                 # Use .loc for safe access using 'Storage' column
                available_storage = node_spec.loc[node_to_check, 'Storage'] - row['Storage']
                # print(f"Retry {retry + 1}, attempt {attempt + 1} for module {row['Module']} on node {node_to_check}.")
                # print(f"Node {node_to_check} has {node_spec.loc[node_to_check, 'Storage']} Storage, app needed {row['Storage']}, available Storage after placement {available_storage}")
            else:
                # print(f"Warning: Node {node_to_check} not found in node_spec index during search.")
                # Treat as failed node? Or rely on random choice moving away?
                # Let's log it as failed and try moving away.
                 if node_to_check not in failed_nodes_for_this_call:
                      failed_nodes_for_this_call.append(node_to_check)
                 # Try to pick a neighbor of the *previous valid node* if possible, or fall back
                 # This part of the original logic might need refinement if node IDs can be invalid often
                 # For now, let's try to move from the last known *good* or attempted node in the path if possible
                 last_known_node = search_path_this_retry[-1] if search_path_this_retry else original_start_node
                 try:
                     temp_resource_id = random.choice(adjacency_dict.get(int(last_known_node), [cloud_node_id])) # Default to cloud if no neighbors
                 except (ValueError, TypeError):
                     temp_resource_id = cloud_node_id # Fallback if ID conversion fails
                 continue # Skip to next attempt

            if available_storage >= 0:
                # print("Suitable placement found.")
                # Return chosen node, remaining Storage, and the list of nodes failed *during this search*
                return node_to_check, available_storage, failed_nodes_for_this_call

            # If here, placement failed on node_to_check due to insufficient Storage
            if node_to_check not in failed_nodes_for_this_call:
                 failed_nodes_for_this_call.append(node_to_check)
            search_path_this_retry.append(node_to_check)

            # Move to a random neighbor for the next attempt (original logic)
            try:
                # Ensure neighbors exist and handle potential errors
                neighbors = adjacency_dict.get(node_to_check, [])
                if not neighbors:
                    # print(f"Node {node_to_check} has no neighbors. Jumping to cloud for next attempt.")
                    temp_resource_id = cloud_node_id # Jump to cloud if no neighbors
                else:
                    temp_resource_id = random.choice(neighbors)
                # print(f"Not enough Storage on {node_to_check}. Trying a new node: {temp_resource_id}")
            except (ValueError, TypeError, KeyError) as e:
                 # print(f"Error finding neighbors for {node_to_check}: {e}. Jumping to cloud.")
                 temp_resource_id = cloud_node_id # Fallback to cloud

        # After 5 attempts, select a new resource_id randomly for the next retry (original logic)
        # Use the last attempted node (temp_resource_id) as the base for the next random hop
        try:
            neighbors = adjacency_dict.get(int(temp_resource_id), [])
            if not neighbors:
                # print(f"Node {temp_resource_id} (end of retry {retry+1}) has no neighbors. Retrying from cloud.")
                current_resource_id = cloud_node_id
            else:
                current_resource_id = random.choice(neighbors)
            # print(f"Retrying with a new node: {current_resource_id} after 5 failed attempts in retry {retry + 1}.")
        except (ValueError, TypeError, KeyError) as e:
            # print(f"Error finding neighbors for {temp_resource_id} before retry: {e}. Retrying from cloud.")
            current_resource_id = cloud_node_id # Fallback to cloud

    # If suitable placement is not found after 5 retries, place on cloud node (original logic)
    # print("Failed to find a suitable edge node after 5 retries. Placing on cloud node.")
    # Return cloud node, its *original* Storage (calling function handles subtraction), and collected failed nodes
    return cloud_node_id, node_spec.loc[cloud_node_id, 'Storage'], failed_nodes_for_this_call

# Modified process_application to handle logging data collection and return TXT lines
# Replaced 'RAM' with 'Storage'
def process_application(app_specifications, source, node_spec, adjacency_dict, cloud_node_id):
    """
    Process each application for placement based on Storage with cloud node fallback.
    Now collects data for TXT logging and returns formatted TXT lines.
    """
    allocation = [] # For JSON output
    placement_line_nodes = [] # For TXT: stores n{id} placements
    required_storage_list = [] # For TXT: stores r{storage} requirements
    all_failed_nodes_for_app = set() # For TXT: stores unique f{id} failed nodes for this app

    # Get required info from source for TXT header
    app_name = source['app']
    source_node_id = source['id_resource']
    lambda_val = source.get('lambda', 'N/A') # Safely get lambda
    module_count = len(app_specifications)

    # print(f"Processing application: {app_name} from source {source_node_id}")
    # Initial resource_id for the *first* module search comes from the source
    current_search_start_node = source_node_id

    for _, row in app_specifications.iterrows():
        module_name = row['Module']
        required_storage = int(row['Storage']) # Assume Storage is integer
        required_storage_list.append(f"r{required_storage}") # Store for TXT spec line

        # Call modified find_suitable_node
        # Pass the node where the *previous* module was placed (or source node initially)
        # as the starting point for the search
        resource_id, available_storage_after_placement, failed_nodes_for_this_call = find_suitable_node(
            row, node_spec, current_search_start_node, adjacency_dict, cloud_node_id
        )

        # Update the set of all failed nodes encountered for this application
        all_failed_nodes_for_app.update(failed_nodes_for_this_call)

        # Update node_spec Storage (original logic adapted)
        # If placed on cloud, available_storage_after_placement is total cloud Storage, so subtract now
        if resource_id == cloud_node_id:
            # print(f"Module {module_name} placed on Cloud ({cloud_node_id}). Updating cloud Storage.")
            node_spec.loc[resource_id, 'Storage'] -= required_storage
        else:
            # print(f"Module {module_name} placed on Node {resource_id}. Updating node Storage.")
            node_spec.loc[resource_id, 'Storage'] = available_storage_after_placement # This was already calculated correctly

        # Record placement for JSON
        placement = {"module_name": module_name, "app": app_name, "id_resource": resource_id}
        allocation.append(placement)

        # Record placement for TXT
        placement_line_nodes.append(f"n{resource_id}")
        # print(f"Module {module_name} placed on node {resource_id}. Available Storage left: {node_spec.loc[resource_id, 'Storage']}")

        # The node where this module was placed becomes the starting point for the next module's search
        current_search_start_node = resource_id

    # --- Format TXT output lines for this application ---
    # Line 1: o{order} m{module_count} l{lambda} r{req1} r{req2}... n{source_node} f{failed1} f{failed2}...
    # Note: 'order' needs to be added by the calling function
    spec_line_parts = [
        f"m{module_count}",
        f"l{lambda_val}"
    ]
    spec_line_parts.extend(required_storage_list) # Use the renamed list
    spec_line_parts.append(f"n{source_node_id}")
    # Add failed nodes, ensuring they are prefixed with 'f' and sorted for consistency
    spec_line_parts.extend(sorted([f"f{int(node_id)}" for node_id in all_failed_nodes_for_app]))
    txt_line1_content = " ".join(spec_line_parts)

    # Line 2: n{placement1} n{placement2}...
    txt_line2_content = " ".join(placement_line_nodes)

    # Return JSON allocation AND the content for the two TXT lines (without the order prefix)
    return allocation, txt_line1_content, txt_line2_content

# Modified create_placementFFNAPS2 to handle TXT file writing
# Replaced 'RAM' with 'Storage'
def create_placementFFNAPS2_Storage(data_pop, adjacency_dict, app_spec, node_spec, popul, app_id, cloud_node_id, save_folder):
    """
    Main function to create placement using FFNAPS2 logic based on Storage.
    Now saves both JSON and appends detailed TXT logs.
    Renamed 'pop' to 'popul' and 'app' to 'app_id' for clarity.
    Added 'save_folder' parameter. Uses 'Storage' instead of 'RAM'.
    (Function name changed slightly to reflect Storage focus).
    """
    JSONfile = {"initialAllocation": []}
    all_txt_output_lines = [] # Collect all TXT lines here
    order = 0  # Initialize order counter for TXT output

    # Create save directory if it doesn't exist
    try:
        os.makedirs(save_folder, exist_ok=True)
    except OSError as e:
        print(f"Error creating directory {save_folder}: {e}")
        return False # Cannot proceed without output directory

    # print("Starting the placement process...")

    # Create a copy of node_spec to modify, preserving the original for potential reuse.
    # This copy will track available 'Storage'.
    local_node_spec = node_spec.copy()

    for source in data_pop['sources']:
        current_app_id = source['app']
        # print(f"\nProcessing source for app: {current_app_id}, starting node: {source['id_resource']}")
        # Ensure app_spec has 'Storage' column
        if 'Storage' not in app_spec.columns:
             print(f"Error: 'Storage' column missing in app_spec. Skipping app {current_app_id}")
             continue
        app_specifications = app_spec[app_spec['app'] == current_app_id].drop_duplicates(subset=['Module', 'app'])

        if app_specifications.empty:
             # print(f"Warning: No modules found for application {current_app_id} in app_spec. Skipping.")
             continue

        # Ensure local_node_spec has 'Storage' column
        if 'Storage' not in local_node_spec.columns:
            print(f"Error: 'Storage' column missing in node_spec copy. Aborting.")
            return False

        # Call the modified process_application (which now uses Storage)
        allocation, txt_line1_content, txt_line2_content = process_application(
            app_specifications, source, local_node_spec, adjacency_dict, cloud_node_id
        )

        if allocation is None: # Check if placement failed catastrophically (shouldn't with cloud fallback)
             # print(f"Placement failed critically for application {current_app_id}")
             continue

        # Add results to overall JSON
        JSONfile["initialAllocation"].extend(allocation)

        # Add formatted lines to TXT output list, prepending the order
        all_txt_output_lines.append(f"o{order} {txt_line1_content}")
        all_txt_output_lines.append(txt_line2_content)

        order += 1 # Increment order for the next application

    # --- Save JSON output ---
    json_filename = f"placement\D100_P{popul}_A{app_id}_firstfit.json" # Adjusted filename
    json_filepath = os.path.join(save_folder, json_filename)
    try:
        with open(json_filepath, "w") as outfile:
            json.dump(JSONfile, outfile, indent=2)
        # print(f"JSON placement saved to {json_filepath}")
    except IOError as e:
        print(f"Error writing JSON file {json_filepath}: {e}")
        # Decide if failure to write JSON should stop TXT writing or return False


    # --- Save TXT output ---
    txt_filename = f"unselected_data\D100_P{popul}_A{app_id}_firstfit.txt" # Adjusted filename
    txt_filepath = os.path.join(save_folder, txt_filename)
    try:
        # Check if file exists and is not empty to decide if newline needed before append
        prepend_newline = os.path.exists(txt_filepath) and os.path.getsize(txt_filepath) > 0
        with open(txt_filepath, "a") as outfile: # Use append mode 'a'
            if prepend_newline:
                outfile.write("\n")
            outfile.write("\n".join(all_txt_output_lines))
        # print(f"TXT log appended to {txt_filepath}")
    except IOError as e:
        print(f"Error appending TXT file {txt_filepath}: {e}")

    # # Assuming checkthisout is a valid function using Storage
    # print(checkthisout(json_filepath, app_spec, node_spec)) # Use json_filepath and ensure checkthisout expects 'Storage'

    # print("\nPlacement process finished.")
    return True


## testing

In [69]:
import json
import time
import pandas as pd

execution_times = []
methodname = "firstfit_withfailure"
hopnumba = 2

pop = 0
app = 10

f = open(fr'C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\topology\100m2.json')
# a dictionary
data = json.load(f)
# print(f)

f = open(fr"C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\population\100m2_P{pop}_E{app}.json")
data_pop = json.load(f)
# print(f)

f = open(fr'C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\application\E{app}_{pop}.json')
data_app = json.load(f)
        # print(f)

new_dict, source = create_dict(data)
app_spec = app_specification(data_app)
node_spec = node_specification(data)
source = list(dict.fromkeys(source))
pop_id = pop
app_id = app

cloud_node_id = 100

save_folder = fr"C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\testing"
# Time the execution of create_placementold
start_time = time.time()
create_placementFFNAPS2_Storage(data_pop, new_dict, app_spec, node_spec, pop_id, app_id, cloud_node_id, save_folder)
end_time = time.time()

execution_time = end_time - start_time
execution_times.append({'pop': pop, 'app': app, 'execution_time': execution_time, 'methodname': methodname})

# Create DataFrame from execution times
df_execution_times = pd.DataFrame(execution_times)

# Save DataFrame to CSV
df_execution_times.to_csv(f"execution_times_{methodname}.csv", index=False)

## runing the loop firstfit

In [105]:
import json
import time
import pandas as pd

execution_times = []
methodname = "firstfit_withfailure"
hopnumba = 2

for pop in range(10):
    for app in range(10):


        f = open(fr'C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\topology\100m2.json')
        # a dictionary
        data = json.load(f)
        # print(f)

        f = open(fr"C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\population\100m2_P{pop}_D100.json")
        data_pop = json.load(f)
        # print(f)

        f = open(fr'C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\application\D100_{app}.json')
        data_app = json.load(f)
        # print(f)

        new_dict, source = create_dict(data)
        app_spec = app_specification(data_app)
        node_spec = node_specification(data)
        source = list(dict.fromkeys(source))
        pop_id = pop
        app_id = app

        cloud_node_id = 100

        save_folder = fr"C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\training_data"
        # Time the execution of create_placementold
        start_time = time.time()
        create_placementFFNAPS2_Storage(data_pop, new_dict, app_spec, node_spec, pop_id, app_id, cloud_node_id, save_folder)
        end_time = time.time()

        execution_time = end_time - start_time
        execution_times.append({'pop': pop, 'app': app, 'execution_time': execution_time, 'methodname': methodname})

# Create DataFrame from execution times
df_execution_times = pd.DataFrame(execution_times)

# Save DataFrame to CSV
df_execution_times.to_csv(f"execution_times_{methodname}.csv", index=False)

# sort_match

In [99]:
import pandas as pd
import json
import random
import os # Added for file path operations

# Modified place_on_fastest_node for Storage and logging
def place_module_fastest_first(row, node_spec, cloud_node_id, cloud_used):
    """
    Places a module based on Storage availability on the fastest nodes first (sorted by Speed).
    Handles cloud persistence and fallback. Tracks failed node attempts for logging.
    Assumes node_spec index is the node 'id'.
    """
    failed_nodes_for_this_call = [] # Track nodes failed in this call

    # Check if cloud persistence forces placement here
    if cloud_used:
        # If any previous module of this app used cloud, force this one to cloud too
        # print(f"Placing Module {row['Module']} on cloud due to previous cloud usage.")
        try:
            required_storage = int(row['Storage'])
            # Directly modify the DataFrame passed in (which should be a copy)
            storage_after_placement = node_spec.loc[cloud_node_id, 'Storage'] - required_storage
            node_spec.loc[cloud_node_id, 'Storage'] = storage_after_placement
            # No new nodes failed in this specific call if forced to cloud initially
            return cloud_node_id, True, []
        except (KeyError, ValueError, TypeError) as e:
            print(f"Error placing module {row['Module']} on forced cloud ({cloud_node_id}): {e}")
            # Decide on fallback? For now, assume cloud placement still happens conceptually
            # Mark cloud as failed? Or return error? Let's return cloud ID but log the issue.
            return cloud_node_id, True, [f"f{cloud_node_id}_error"] # Log error on cloud node

    # If not forced to cloud, try fastest nodes first
    try:
        required_storage = int(row['Storage'])

        # Sort available *non-cloud* nodes by 'Speed' (Instruction Per Tick?) descending
        # Assumes 'Speed' column exists. Filter using index now.
        non_cloud_nodes = node_spec[node_spec.index != cloud_node_id]
        # Handle case where no non-cloud nodes exist
        if non_cloud_nodes.empty:
             # print(f"No non-cloud nodes available. Placing Module {row['Module']} on cloud.")
             # Fall through to cloud placement logic below
             pass # Allow logic to proceed to cloud fallback naturally
        else:
            sorted_node_ids = non_cloud_nodes.sort_values('Speed', ascending=False).index

            for node_id in sorted_node_ids:
                try:
                    available_storage = node_spec.loc[node_id, 'Storage'] - required_storage
                    if available_storage >= 0:
                        # print(f"Module {row['Module']} placed on fast node {node_id}.")
                        # Update the DataFrame copy
                        node_spec.loc[node_id, 'Storage'] = available_storage
                        # Return node, cloud_used=False, and nodes failed *before* this success
                        return node_id, False, failed_nodes_for_this_call
                    else:
                        # Insufficient storage on this node, log it as failed for this attempt
                        if node_id not in failed_nodes_for_this_call:
                            failed_nodes_for_this_call.append(node_id)
                except (KeyError, ValueError, TypeError) as e:
                     # Error accessing this specific node, log it as failed
                     print(f"Error checking node {node_id} for module {row['Module']}: {e}")
                     if node_id not in failed_nodes_for_this_call:
                            failed_nodes_for_this_call.append(f"{node_id}_error") # Append error marker

        # If loop completes without returning, no suitable *non-cloud* node was found
        # print(f"No suitable fast node found for {row['Module']}. Placing on cloud.")
        # Update failed list: all checked non-cloud nodes failed
        # Use the sorted list we tried, filter out any potential error markers already added
        all_checked_ids = [nid for nid in sorted_node_ids if isinstance(nid, (int, float))] # Ensure numeric IDs
        failed_nodes_for_this_call = all_checked_ids # All checked nodes failed

    except (KeyError, ValueError, TypeError) as e:
        print(f"Error processing module {row['Module']} requirements or node sorting: {e}. Falling back to cloud.")
        # Treat as if all non-cloud nodes failed if error happened before/during sorting
        try:
             failed_nodes_for_this_call = list(node_spec[node_spec.index != cloud_node_id].index)
        except: # Catch all if node_spec itself is problematic
             failed_nodes_for_this_call = ["f_error_processing_nodes"]


    # --- Cloud Placement Fallback ---
    try:
        required_storage = int(row['Storage']) # Recalculate in case of error above
        storage_after_placement = node_spec.loc[cloud_node_id, 'Storage'] - required_storage
        # Update the DataFrame copy
        node_spec.loc[cloud_node_id, 'Storage'] = storage_after_placement
        return cloud_node_id, True, failed_nodes_for_this_call # Return cloud, set flag, return failed nodes
    except (KeyError, ValueError, TypeError) as e:
        print(f"CRITICAL Error placing module {row['Module']} on fallback cloud ({cloud_node_id}): {e}")
        # If cloud itself fails, something is very wrong. Return cloud ID but flag error.
        return cloud_node_id, True, failed_nodes_for_this_call + [f"f{cloud_node_id}_critical_error"]


# Modified process_application_fastest_nodes for Storage and logging
def process_application_fastest_nodes(app_specifications, source, node_spec, cloud_node_id):
    """
    Processes modules of an application using the 'fastest node first' strategy based on Storage.
    Collects data for TXT logging and returns formatted TXT lines.
    Modifies the passed node_spec (which should be a copy).
    """
    allocation = [] # For JSON output
    placement_line_nodes = [] # For TXT: stores n{id} placements
    required_storage_list = [] # For TXT: stores r{storage} requirements
    all_failed_nodes_for_app = set() # For TXT: stores unique f{id} failed nodes for this app

    # Get required info from source for TXT header
    app_name = source['app']
    source_node_id = source['id_resource'] # Keep track of original source for logging
    lambda_val = source.get('lambda', 'N/A') # Safely get lambda
    module_count = len(app_specifications)

    cloud_used = False  # Flag to track if *any* module of *this app* is placed on cloud

    # print(f"Processing application: {app_name} from source {source_node_id} (Fastest Node Strategy)")

    for _, row in app_specifications.iterrows():
        module_name = row['Module']
        try:
            required_storage = int(row['Storage']) # Assume Storage is integer
            required_storage_list.append(f"r{required_storage}") # Store for TXT spec line
        except (KeyError, ValueError, TypeError):
             print(f"Warning: Invalid Storage for module {module_name} (App {app_name}). Skipping module.")
             required_storage_list.append("r?_invalid")
             placement_line_nodes.append("n?_skipped")
             # Should we mark cloud used? Let's assume not for a skipped module.
             continue # Skip to next module

        # Call modified placement function
        resource_id, cloud_used_flag_updated, failed_nodes_for_this_call = place_module_fastest_first(
            row, node_spec, cloud_node_id, cloud_used
        )

        # Update the overall cloud_used flag for the application
        cloud_used = cloud_used_flag_updated

        # Update the set of all failed nodes encountered for this application
        # Ensure we handle potential string markers like 'id_error'
        for node in failed_nodes_for_this_call:
             all_failed_nodes_for_app.add(node) # Add directly, set handles uniqueness

        # Record placement for JSON
        placement = {"module_name": module_name, "app": app_name, "id_resource": resource_id}
        allocation.append(placement)

        # Record placement for TXT
        placement_line_nodes.append(f"n{resource_id}")
        # print(f"Module {module_name} placed on node {resource_id}. Cloud used so far for app: {cloud_used}")


    # --- Format TXT output lines for this application ---
    # Line 1: o{order} m{module_count} l{lambda} r{req1} r{req2}... n{source_node} f{failed1} f{failed2}...
    spec_line_parts = [
        f"m{module_count}",
        f"l{lambda_val}"
    ]
    spec_line_parts.extend(required_storage_list)
    spec_line_parts.append(f"n{source_node_id}") # Use original source node ID
    # Add failed nodes, prefixing numeric IDs with 'f', keeping error strings as is, and sorting
    formatted_failed_nodes = []
    for node in all_failed_nodes_for_app:
        try:
            # Try converting to int to check if it's a pure node ID
            int_node = int(node)
            formatted_failed_nodes.append(f"f{int_node}")
        except (ValueError, TypeError):
            # If conversion fails, it's likely an error string like 'id_error' or 'fX_error'
            formatted_failed_nodes.append(str(node)) # Keep the error string
    spec_line_parts.extend(sorted(formatted_failed_nodes))
    txt_line1_content = " ".join(spec_line_parts)

    # Line 2: n{placement1} n{placement2}...
    txt_line2_content = " ".join(placement_line_nodes)

    # Return JSON allocation AND the content for the two TXT lines (without the order prefix)
    return allocation, txt_line1_content, txt_line2_content


# Modified create_placementTanejaModified for Storage and logging
def create_placementTanejaModified(data_pop, app_spec, node_spec, popul, app_id, cloud_node_id, save_folder):
    """
    Main function to create placement using Taneja Modified (Fastest Node First) logic based on Storage.
    Saves both JSON and appends detailed TXT logs.
    Uses 'Storage' instead of 'RAM'. Assumes 'Speed' column exists in node_spec.
    Removed unused 'new_dict' parameter.
    """
    JSONfile = {"initialAllocation": []}
    all_txt_output_lines = [] # Collect all TXT lines here
    order = 0  # Initialize order counter for TXT output

    # Create save directory if it doesn't exist
    try:
        os.makedirs(save_folder, exist_ok=True)
    except OSError as e:
        print(f"Error creating directory {save_folder}: {e}")
        return False # Cannot proceed without output directory

    # print("Starting the Taneja Modified (fastest first) placement process...")

    # --- Create a local copy of node_spec to modify ---
    # Ensure the index is 'id' for consistent access in helper functions
    local_node_spec = node_spec.copy()
    if 'id' not in local_node_spec.columns and not pd.api.types.is_integer_dtype(local_node_spec.index):
         print("Error: node_spec must have an 'id' column or an integer index.")
         return False
    if 'id' in local_node_spec.columns and not pd.api.types.is_integer_dtype(local_node_spec.index):
        local_node_spec = local_node_spec.set_index('id', drop=False) # Keep id column if needed elsewhere too

    # Ensure required columns exist
    if 'Storage' not in local_node_spec.columns:
        print("Error: 'Storage' column missing in node_spec.")
        return False
    if 'Speed' not in local_node_spec.columns:
        print("Error: 'Speed' column missing in node_spec (required for sorting).")
        return False
    if 'Storage' not in app_spec.columns:
        print("Error: 'Storage' column missing in app_spec.")
        return False


    for source in data_pop['sources']:
        current_app_id = source['app']
        # print(f"\nProcessing source for app: {current_app_id}, initial source node: {source['id_resource']}")
        app_specifications = app_spec[app_spec['app'] == current_app_id].drop_duplicates(subset=['Module', 'app'])

        if app_specifications.empty:
             # print(f"Warning: No modules found for application {current_app_id} in app_spec. Skipping.")
             continue

        # Call the modified process_application function, passing the local copy
        allocation, txt_line1_content, txt_line2_content = process_application_fastest_nodes(
            app_specifications, source, local_node_spec, cloud_node_id
        )

        if allocation is None: # Should not happen given the logic, but check anyway
             print(f"Placement failed critically for application {current_app_id}")
             continue

        # Add results to overall JSON
        JSONfile["initialAllocation"].extend(allocation)

        # Add formatted lines to TXT output list, prepending the order
        all_txt_output_lines.append(f"o{order} {txt_line1_content}")
        all_txt_output_lines.append(txt_line2_content)

        order += 1 # Increment order for the next application

    # --- Save JSON output ---
    json_filename = f"placement\D100_P{popul}_A{app_id}_sortmatch.json" # Adjusted filename
    json_filepath = os.path.join(save_folder, json_filename)
    try:
        with open(json_filepath, "w") as outfile:
            json.dump(JSONfile, outfile, indent=2)
        # print(f"JSON placement saved to {json_filepath}")
    except IOError as e:
        print(f"Error writing JSON file {json_filepath}: {e}")

    # --- Save TXT output ---
    txt_filename = f"D100_P{popul}_A{app_id}_sortmatch.txt" # Adjusted filename
    txt_filepath = os.path.join(save_folder, txt_filename)
    try:
        # Check if file exists and is not empty to decide if newline needed before append
        prepend_newline = os.path.exists(txt_filepath) and os.path.getsize(txt_filepath) > 0
        with open(txt_filepath, "a") as outfile: # Use append mode 'a'
            if prepend_newline:
                outfile.write("\n")
            outfile.write("\n".join(all_txt_output_lines))
        # print(f"TXT log appended to {txt_filepath}")
    except IOError as e:
        print(f"Error appending TXT file {txt_filepath}: {e}")

    # print("\nPlacement process finished.")
    return True

# --- Example Usage Setup (Updated for Storage and Speed) ---
# Example DataFrames and Dictionaries (replace with your actual data loading)
# node_spec = pd.DataFrame({
#     'id': [1, 2, 3, 100],
#     'Storage': [1000, 800, 1200, 10000], # Using Storage
#     'Speed': [500, 1500, 1000, 100] # Added Speed (higher is faster?)
# }).set_index('id') # Set 'id' as index
# app_spec = pd.DataFrame({
#     'app': ['App1', 'App1', 'App2'],
#     'Module': ['M1', 'M2', 'M3'],
#     'Storage': [400, 500, 700] # Using Storage
# })
# data_pop = {
#     'sources': [
#         {'app': 'App1', 'id_resource': 1, 'lambda': 0.5}, # Source node isn't directly used by placement logic here
#         {'app': 'App2', 'id_resource': 2, 'lambda': 0.8}
#     ]
# }
# # adjacency_dict (new_dict) is not used by this logic, so removed from params
# cloud_node_id = 100
# popul = 1
# app_id = "SetB"
# tipe = "FastestTestRun"
# save_folder = "fastest_storage_output" # Adjusted output folder name

# --- Call the main function ---
# Ensure your node_spec and app_spec DataFrames have 'Storage' columns and node_spec has 'Speed'
# success = create_placementTanejaModified_Storage(data_pop, app_spec, node_spec, popul, app_id, tipe, cloud_node_id, save_folder)
# print(f"Placement successful: {success}")

## testing_dummy

In [78]:
import json
import time
import pandas as pd

execution_times = []
methodname = "sortmatch"
hopnumba = 2

pop = 0
app = 10

f = open(fr'C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\topology\100m2.json')
# a dictionary
data = json.load(f)
# print(f)

f = open(fr"C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\population\100m2_P{pop}_E{app}.json")
data_pop = json.load(f)
# print(f)

f = open(fr'C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\application\E{app}_{pop}.json')
data_app = json.load(f)
        # print(f)

new_dict, source = create_dict(data)
app_spec = app_specification(data_app)
node_spec = node_specification(data)
source = list(dict.fromkeys(source))
pop_id = pop
app_id = app

cloud_node_id = 100

save_folder = fr"C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\testing"
# Time the execution of create_placementold
start_time = time.time()
create_placementTanejaModified(data_pop, app_spec, node_spec, pop_id, app_id, cloud_node_id, save_folder)
end_time = time.time()

execution_time = end_time - start_time
execution_times.append({'pop': pop, 'app': app, 'execution_time': execution_time, 'methodname': methodname})

# Create DataFrame from execution times
df_execution_times = pd.DataFrame(execution_times)

# Save DataFrame to CSV
df_execution_times.to_csv(f"execution_times_{methodname}.csv", index=False)

## running the loop sortmatch

In [100]:
import json
import time
import pandas as pd

execution_times = []
methodname = "sortmatch_withfailure"
hopnumba = 2

for pop in range(10):
    for app in range(10):


        f = open(fr'C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\topology\100m2.json')
        # a dictionary
        data = json.load(f)
        # print(f)

        f = open(fr"C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\population\100m2_P{pop}_D100.json")
        data_pop = json.load(f)
        # print(f)

        f = open(fr'C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\application\D100_{app}.json')
        data_app = json.load(f)
        # print(f)

        new_dict, source = create_dict(data)
        app_spec = app_specification(data_app)
        node_spec = node_specification(data)
        source = list(dict.fromkeys(source))
        pop_id = pop
        app_id = app

        cloud_node_id = 100

        save_folder = fr"C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\training_data"
        # Time the execution of create_placementold
        start_time = time.time()
        create_placementTanejaModified(data_pop, app_spec, node_spec, pop_id, app_id, cloud_node_id, save_folder)
        end_time = time.time()

        execution_time = end_time - start_time
        execution_times.append({'pop': pop, 'app': app, 'execution_time': execution_time, 'methodname': methodname})

# Create DataFrame from execution times
df_execution_times = pd.DataFrame(execution_times)

# Save DataFrame to CSV
df_execution_times.to_csv(f"execution_times_{methodname}.csv", index=False)

# skarlt_firstfit

In [101]:
import pandas as pd
import json
import random
import os

# Modified find_suitable_node to return failed attempts for logging
# Replaced 'RAM' with 'Storage'
def find_suitable_node(row, node_spec, current_resource_id, adjacency_dict, cloud_node_id):
    """
    Find a suitable node for module placement based on Storage with a fallback to cloud node.
    Now also returns a list of nodes that failed due to insufficient Storage during this search.
    """
    failed_nodes_for_this_call = [] # Track nodes that failed in this specific search
    original_start_node = current_resource_id # Keep track of where this search started

    for retry in range(5):  # retry up to 5 times
        search_path_this_retry = [] # Track path within a retry to avoid adding duplicates quickly
        temp_resource_id = original_start_node if retry == 0 else current_resource_id # Start fresh or continue from last random hop

        for attempt in range(5):  # try up to 5 times per retry
            available_storage = -1 # Default to -1 if node ID is invalid or not found
            node_to_check = int(temp_resource_id) # Ensure integer ID

            if node_to_check in node_spec.index:
                 # Use .loc for safe access using 'Storage' column
                available_storage = node_spec.loc[node_to_check, 'Storage'] - row['Storage']
                # print(f"Retry {retry + 1}, attempt {attempt + 1} for module {row['Module']} on node {node_to_check}.")
                # print(f"Node {node_to_check} has {node_spec.loc[node_to_check, 'Storage']} Storage, app needed {row['Storage']}, available Storage after placement {available_storage}")
            else:
                # print(f"Warning: Node {node_to_check} not found in node_spec index during search.")
                # Treat as failed node? Or rely on random choice moving away?
                # Let's log it as failed and try moving away.
                 if node_to_check not in failed_nodes_for_this_call:
                      failed_nodes_for_this_call.append(node_to_check)
                 # Try to pick a neighbor of the *previous valid node* if possible, or fall back
                 # This part of the original logic might need refinement if node IDs can be invalid often
                 # For now, let's try to move from the last known *good* or attempted node in the path if possible
                 last_known_node = search_path_this_retry[-1] if search_path_this_retry else original_start_node
                 try:
                     temp_resource_id = random.choice(adjacency_dict.get(int(last_known_node), [cloud_node_id])) # Default to cloud if no neighbors
                 except (ValueError, TypeError):
                     temp_resource_id = cloud_node_id # Fallback if ID conversion fails
                 continue # Skip to next attempt

            if available_storage >= 0:
                # print("Suitable placement found.")
                # Return chosen node, remaining Storage, and the list of nodes failed *during this search*
                return node_to_check, available_storage, failed_nodes_for_this_call

            # If here, placement failed on node_to_check due to insufficient Storage
            if node_to_check not in failed_nodes_for_this_call:
                 failed_nodes_for_this_call.append(node_to_check)
            search_path_this_retry.append(node_to_check)

            # Move to a random neighbor for the next attempt (original logic)
            try:
                # Ensure neighbors exist and handle potential errors
                neighbors = adjacency_dict.get(node_to_check, [])
                if not neighbors:
                    # print(f"Node {node_to_check} has no neighbors. Jumping to cloud for next attempt.")
                    temp_resource_id = cloud_node_id # Jump to cloud if no neighbors
                else:
                    temp_resource_id = random.choice(neighbors)
                # print(f"Not enough Storage on {node_to_check}. Trying a new node: {temp_resource_id}")
            except (ValueError, TypeError, KeyError) as e:
                 # print(f"Error finding neighbors for {node_to_check}: {e}. Jumping to cloud.")
                 temp_resource_id = cloud_node_id # Fallback to cloud

        # After 5 attempts, select a new resource_id randomly for the next retry (original logic)
        # Use the last attempted node (temp_resource_id) as the base for the next random hop
        try:
            neighbors = adjacency_dict.get(int(temp_resource_id), [])
            if not neighbors:
                # print(f"Node {temp_resource_id} (end of retry {retry+1}) has no neighbors. Retrying from cloud.")
                current_resource_id = cloud_node_id
            else:
                current_resource_id = random.choice(neighbors)
            # print(f"Retrying with a new node: {current_resource_id} after 5 failed attempts in retry {retry + 1}.")
        except (ValueError, TypeError, KeyError) as e:
            # print(f"Error finding neighbors for {temp_resource_id} before retry: {e}. Retrying from cloud.")
            current_resource_id = cloud_node_id # Fallback to cloud

    # If suitable placement is not found after 5 retries, place on cloud node (original logic)
    # print("Failed to find a suitable edge node after 5 retries. Placing on cloud node.")
    # Return cloud node, its *original* Storage (calling function handles subtraction), and collected failed nodes
    return cloud_node_id, node_spec.loc[cloud_node_id, 'Storage'], failed_nodes_for_this_call

# Modified process_application to handle logging data collection and return TXT lines
# Replaced 'RAM' with 'Storage'
def process_application(app_specifications, source, node_spec, adjacency_dict, cloud_node_id):
    """
    Process each application for placement based on Storage with cloud node fallback.
    Now collects data for TXT logging and returns formatted TXT lines.
    """
    allocation = [] # For JSON output
    placement_line_nodes = [] # For TXT: stores n{id} placements
    required_storage_list = [] # For TXT: stores r{storage} requirements
    all_failed_nodes_for_app = set() # For TXT: stores unique f{id} failed nodes for this app

    # Get required info from source for TXT header
    app_name = source['app']
    source_node_id = source['id_resource']
    lambda_val = source.get('lambda', 'N/A') # Safely get lambda
    module_count = len(app_specifications)

    # print(f"Processing application: {app_name} from source {source_node_id}")
    # Initial resource_id for the *first* module search comes from the source
    current_search_start_node = source_node_id

    for _, row in app_specifications.iterrows():
        module_name = row['Module']
        required_storage = int(row['Storage']) # Assume Storage is integer
        required_storage_list.append(f"r{required_storage}") # Store for TXT spec line

        # Call modified find_suitable_node
        # Pass the node where the *previous* module was placed (or source node initially)
        # as the starting point for the search
        resource_id, available_storage_after_placement, failed_nodes_for_this_call = find_suitable_node(
            row, node_spec, current_search_start_node, adjacency_dict, cloud_node_id
        )

        # Update the set of all failed nodes encountered for this application
        all_failed_nodes_for_app.update(failed_nodes_for_this_call)

        # Update node_spec Storage (original logic adapted)
        # If placed on cloud, available_storage_after_placement is total cloud Storage, so subtract now
        if resource_id == cloud_node_id:
            # print(f"Module {module_name} placed on Cloud ({cloud_node_id}). Updating cloud Storage.")
            node_spec.loc[resource_id, 'Storage'] -= required_storage
        else:
            # print(f"Module {module_name} placed on Node {resource_id}. Updating node Storage.")
            node_spec.loc[resource_id, 'Storage'] = available_storage_after_placement # This was already calculated correctly

        # Record placement for JSON
        placement = {"module_name": module_name, "app": app_name, "id_resource": resource_id}
        allocation.append(placement)

        # Record placement for TXT
        placement_line_nodes.append(f"n{resource_id}")
        # print(f"Module {module_name} placed on node {resource_id}. Available Storage left: {node_spec.loc[resource_id, 'Storage']}")

        # The node where this module was placed becomes the starting point for the next module's search
        current_search_start_node = resource_id

    # --- Format TXT output lines for this application ---
    # Line 1: o{order} m{module_count} l{lambda} r{req1} r{req2}... n{source_node} f{failed1} f{failed2}...
    # Note: 'order' needs to be added by the calling function
    spec_line_parts = [
        f"m{module_count}",
        f"l{lambda_val}"
    ]
    spec_line_parts.extend(required_storage_list) # Use the renamed list
    spec_line_parts.append(f"n{source_node_id}")
    # Add failed nodes, ensuring they are prefixed with 'f' and sorted for consistency
    spec_line_parts.extend(sorted([f"f{int(node_id)}" for node_id in all_failed_nodes_for_app]))
    txt_line1_content = " ".join(spec_line_parts)

    # Line 2: n{placement1} n{placement2}...
    txt_line2_content = " ".join(placement_line_nodes)

    # Return JSON allocation AND the content for the two TXT lines (without the order prefix)
    return allocation, txt_line1_content, txt_line2_content

# Modified create_placementFFNAPS2 to handle TXT file writing
# Replaced 'RAM' with 'Storage'
def create_placementskarlatFF_Storage(data_pop, adjacency_dict, app_spec, node_spec, popul, app_id, cloud_node_id, save_folder):
    """
    Main function to create placement using FFNAPS2 logic based on Storage.
    Now saves both JSON and appends detailed TXT logs.
    Renamed 'pop' to 'popul' and 'app' to 'app_id' for clarity.
    Added 'save_folder' parameter. Uses 'Storage' instead of 'RAM'.
    (Function name changed slightly to reflect Storage focus).
    """
    JSONfile = {"initialAllocation": []}
    all_txt_output_lines = [] # Collect all TXT lines here
    order = 0  # Initialize order counter for TXT output

    # Create save directory if it doesn't exist
    try:
        os.makedirs(save_folder, exist_ok=True)
    except OSError as e:
        print(f"Error creating directory {save_folder}: {e}")
        return False # Cannot proceed without output directory

    # print("Starting the placement process...")

    # Create a copy of node_spec to modify, preserving the original for potential reuse.
    # This copy will track available 'Storage'.
    local_node_spec = node_spec.copy()

    for source in data_pop['sources']:
        current_app_id = source['app']
        # print(f"\nProcessing source for app: {current_app_id}, starting node: {source['id_resource']}")
        # Ensure app_spec has 'Storage' column
        if 'Storage' not in app_spec.columns:
             print(f"Error: 'Storage' column missing in app_spec. Skipping app {current_app_id}")
             continue
        app_specifications = app_spec[app_spec['app'] == current_app_id].drop_duplicates(subset=['Module', 'app'])

        if app_specifications.empty:
             # print(f"Warning: No modules found for application {current_app_id} in app_spec. Skipping.")
             continue

        # Ensure local_node_spec has 'Storage' column
        if 'Storage' not in local_node_spec.columns:
            print(f"Error: 'Storage' column missing in node_spec copy. Aborting.")
            return False

        # Call the modified process_application (which now uses Storage)
        allocation, txt_line1_content, txt_line2_content = process_application(
            app_specifications, source, local_node_spec, adjacency_dict, cloud_node_id
        )

        if allocation is None: # Check if placement failed catastrophically (shouldn't with cloud fallback)
             # print(f"Placement failed critically for application {current_app_id}")
             continue

        # Add results to overall JSON
        JSONfile["initialAllocation"].extend(allocation)

        # Add formatted lines to TXT output list, prepending the order
        all_txt_output_lines.append(f"o{order} {txt_line1_content}")
        all_txt_output_lines.append(txt_line2_content)

        order += 1 # Increment order for the next application

    # --- Save JSON output ---
    json_filename = f"placement\D100_P{popul}_A{app_id}_clusterfirstfit.json" # Adjusted filename
    json_filepath = os.path.join(save_folder, json_filename)
    try:
        with open(json_filepath, "w") as outfile:
            json.dump(JSONfile, outfile, indent=2)
        # print(f"JSON placement saved to {json_filepath}")
    except IOError as e:
        print(f"Error writing JSON file {json_filepath}: {e}")
        # Decide if failure to write JSON should stop TXT writing or return False


    # --- Save TXT output ---
    txt_filename = f"D100_P{popul}_A{app_id}_clusterfirstfit.txt" # Adjusted filename
    txt_filepath = os.path.join(save_folder, txt_filename)
    try:
        # Check if file exists and is not empty to decide if newline needed before append
        prepend_newline = os.path.exists(txt_filepath) and os.path.getsize(txt_filepath) > 0
        with open(txt_filepath, "a") as outfile: # Use append mode 'a'
            if prepend_newline:
                outfile.write("\n")
            outfile.write("\n".join(all_txt_output_lines))
        # print(f"TXT log appended to {txt_filepath}")
    except IOError as e:
        print(f"Error appending TXT file {txt_filepath}: {e}")

    # # Assuming checkthisout is a valid function using Storage
    # print(checkthisout(json_filepath, app_spec, node_spec)) # Use json_filepath and ensure checkthisout expects 'Storage'

    # print("\nPlacement process finished.")
    return True


In [102]:
def create_map_skarlat(topo):
    new_dict = dict()
    source = []

    for i in topo['links']:
        source_node = i['source']
        target_node = i['target']
        source_type = topo['nodes'][source_node]['label']
        target_type = topo['nodes'][target_node]['label']

        # Rule for gateway: connect only to nodes (not node heads)
        if source_type == 'gateway':
            new_dict = add_connection(new_dict, source_node, target_node)
            source.append(source_node)

        # Rule for nodes: connect to other nodes and node heads
        elif source_type != 'gateway' and source_type != 'cloud':
            new_dict = add_connection(new_dict, source_node, target_node)
            if target_type != 'gateway' and target_type != 'cloud':
                new_dict = add_connection(new_dict, target_node, source_node)
            source.append(source_node)

        # Rule for node heads: connect only to other node heads
        elif source_type == 'head_node' and target_type == 'head_node':
            new_dict = add_connection(new_dict, source_node, target_node)
            new_dict = add_connection(new_dict, target_node, source_node)
            source.append(source_node)

    for i in topo['links']:
        source_node = i['target']
        target_node = i['source']
        source_type = topo['nodes'][source_node]['label']
        target_type = topo['nodes'][target_node]['label']

        # Rule for gateway: connect only to nodes (not node heads)
        if source_type == 'gateway':
            new_dict = add_connection(new_dict, source_node, target_node)
            source.append(source_node)

        # Rule for nodes: connect to other nodes and node heads
        elif source_type != 'gateway' and source_type != 'cloud':
            new_dict = add_connection(new_dict, source_node, target_node)
            if target_type != 'gateway' and target_type != 'cloud':
                new_dict = add_connection(new_dict, target_node, source_node)
            source.append(source_node)

        # Rule for node heads: connect only to other node heads
        elif source_type == 'head_node' and target_type == 'head_node':
            new_dict = add_connection(new_dict, source_node, target_node)
            new_dict = add_connection(new_dict, target_node, source_node)
            source.append(source_node)



    return new_dict, source

def add_connection(graph_dict, src, tgt):
    """ Helper function to add a connection to the graph dictionary """
    if src in graph_dict:
        graph_dict[src].append(tgt)
    else:
        graph_dict[src] = [tgt]
    return graph_dict


## testing clusterfirstfit

In [ ]:
import json
import time
import pandas as pd

execution_times = []
methodname = "clusterfirstfit"
hopnumba = 2

pop = 0
app = 10

f = open(fr'C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\topology\100m2.json')
# a dictionary
data = json.load(f)
# print(f)

f = open(fr"C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\population\100m2_P{pop}_E{app}.json")
data_pop = json.load(f)
# print(f)

f = open(fr'C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\application\E{app}_{pop}.json')
data_app = json.load(f)
        # print(f)

new_dict, source = create_map_skarlat(data)
app_spec = app_specification(data_app)
node_spec = node_specification(data)
source = list(dict.fromkeys(source))
pop_id = pop
app_id = app

cloud_node_id = 100

save_folder = fr"C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\testing"
# Time the execution of create_placementold
start_time = time.time()
create_placementskarlatFF_Storage(data_pop, new_dict, app_spec, node_spec, pop_id, app_id, cloud_node_id, save_folder)
end_time = time.time()

execution_time = end_time - start_time
execution_times.append({'pop': pop, 'app': app, 'execution_time': execution_time, 'methodname': methodname})

# Create DataFrame from execution times
df_execution_times = pd.DataFrame(execution_times)

# Save DataFrame to CSV
df_execution_times.to_csv(f"execution_times_{methodname}.csv", index=False)

## running the loop clusterfirstfit

In [104]:
import json
import time
import pandas as pd

execution_times = []
methodname = "clusterfirstfit_withfailure"
hopnumba = 2

for pop in range(10):
    for app in range(10):


        f = open(fr'C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\topology\100m2.json')
        # a dictionary
        data = json.load(f)
        # print(f)

        f = open(fr"C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\population\100m2_P{pop}_D100.json")
        data_pop = json.load(f)
        # print(f)

        f = open(fr'C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\application\D100_{app}.json')
        data_app = json.load(f)
        # print(f)

        new_dict, source = create_map_skarlat(data)
        app_spec = app_specification(data_app)
        node_spec = node_specification(data)
        source = list(dict.fromkeys(source))
        pop_id = pop
        app_id = app

        cloud_node_id = 100

        save_folder = fr"C:\Users\komputer\OneDrive\Documents\GitHub\MOFPSS-Seq2seq\evaluation_files\training_data"
        # Time the execution of create_placementold
        start_time = time.time()
        create_placementskarlatFF_Storage(data_pop, new_dict, app_spec, node_spec, pop_id, app_id, cloud_node_id, save_folder)
        end_time = time.time()

        execution_time = end_time - start_time
        execution_times.append({'pop': pop, 'app': app, 'execution_time': execution_time, 'methodname': methodname})

# Create DataFrame from execution times
df_execution_times = pd.DataFrame(execution_times)

# Save DataFrame to CSV
df_execution_times.to_csv(f"execution_times_{methodname}.csv", index=False)